In [ ]:
import zipfile
import os

# Replace with your actual filename if different
zip_path = "/content/Mango_Banana_Dataset.zip"
extract_path = "/content/data"

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Top-level contents after extraction:")
print(os.listdir(extract_path))

In [ ]:
# This script trains a mango classifier (Raw, Ripe, Not a Mango) with data augmentation, fine-tuning,
# and saves the model as TFLite. It also evaluates the test accuracy.

import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array

# --- Parameters ---
TRAIN_PATH = "/content/data/data/train/images"
TEST_PATH = "/content/data/data/test/images"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
CLASS_NAMES = ["Raw_Mango", "Ripe_Mango", "Not_a_Mango"]

# --- Labeling from filename ---
def label_from_filename(filename):
    fname = filename.lower()
    if "raw_mango" in fname:
        return 0
    elif "ripe_mango" in fname:
        return 1
    elif "ripe_banana" in fname or "raw_banana" in fname:
        return 2
    else:
        return None

# --- Load and label training data ---
image_paths = [os.path.join(TRAIN_PATH, fname) for fname in os.listdir(TRAIN_PATH) if fname.endswith(('.jpg', '.png'))]
labeled_paths = [(p, label_from_filename(p)) for p in image_paths if label_from_filename(p) is not None]

# --- Balance classes ---
class_paths = {0: [], 1: [], 2: []}
for path, label in labeled_paths:
    class_paths[label].append(path)

min_count = min(len(paths) for paths in class_paths.values())
balanced_paths = []
for label in class_paths:
    selected = random.sample(class_paths[label], min_count)
    balanced_paths.extend([(p, label) for p in selected])

random.shuffle(balanced_paths)
print(f"✅ Balanced dataset to {min_count} samples per class")
print(f"✅ Total training samples: {len(balanced_paths)}")

# --- Load and preprocess training images ---
def preprocess_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    img = img_to_array(img) / 255.0
    return img

X_train = np.array([preprocess_image(p) for p, _ in balanced_paths])
y_train = np.array([label for _, label in balanced_paths])

# --- Data augmentation ---
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

# --- Build model ---
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
base_model.trainable = True

# Unfreeze only top 50 layers
for layer in base_model.layers[:-50]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(len(CLASS_NAMES), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- Train model ---
print("🚀 Training mango classifier (no validation set)...")
model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
          epochs=EPOCHS,
          steps_per_epoch=len(X_train)//BATCH_SIZE,
          verbose=1)

# --- Save as TFLite model ---
print(" Converting to TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("mango_classifier_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as: mango_classifier_model.tflite")

# --- Evaluate on training set ---
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f" Final Training Accuracy: {train_acc * 100:.2f}%")

# --- Load and preprocess test images ---
test_paths = [os.path.join(TEST_PATH, fname) for fname in os.listdir(TEST_PATH) if fname.endswith(('.jpg', '.png'))]
test_data = [(p, label_from_filename(p)) for p in test_paths if label_from_filename(p) is not None]

X_test = np.array([preprocess_image(p) for p, _ in test_data])
y_test = np.array([label for _, label in test_data])

# --- Evaluate on test set ---
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f" Final Test Accuracy: {test_acc * 100:.2f}%")


In [ ]:
# This script loads your mango_classifier_model.tflite and classifies all unique images, showing predictions visually and printing confidence.

from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# --- Load the TFLite model ---
interpreter = tf.lite.Interpreter(model_path="mango_classifier_model.tflite")
interpreter.allocate_tensors()

# --- Get input/output details ---
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- Class names used during training ---
class_names = ["Raw_Mango", "Ripe_Mango", "Not_a_Mango"]

# --- Preprocessing function ---
def preprocess_image(img_path):
    img = Image.open(img_path).resize((224, 224)).convert("RGB")
    img_arr = np.array(img) / 255.0
    img_arr = img_arr.astype(np.float32)
    img_arr = np.expand_dims(img_arr, axis=0)
    return img, img_arr

# --- Prediction and image display ---
def classify_and_show(img_path):
    try:
        img, img_tensor = preprocess_image(img_path)

        interpreter.set_tensor(input_details[0]['index'], img_tensor)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]

        pred_idx = np.argmax(output)
        pred_label = class_names[pred_idx]
        confidence = output[pred_idx]

        #  Show image with prediction
        plt.imshow(img)
        plt.title(f"{pred_label} ({confidence:.2f})")
        plt.axis('off')
        plt.show()

        # Console output
        print(f" {img_path}")
        print(f"➡ Prediction: {pred_label} (Confidence: {confidence:.2f})\n")

    except Exception as e:
        print(f" Error with image {img_path}: {e}")

# --- All image paths (deduplicated) ---
test_images = list(set([
    "/content/ripebanana1.jpg",
    "/content/raw_mango.jpg",
    "/content/raw-mango2.webp",
    "/content/bunch_mango.jpg",
    "/content/bunch_banana.jpg",
    "/content/RW_MG5.jpg",
    "/content/RP_MG2.jpg",
    "/content/ripemango.jpg"
]))

# --- Run predictions on all test images ---
for path in test_images:
    classify_and_show(path)

In [ ]:
#BANANA CLASSIFER WITH ACURACY
import os
import shutil
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Dataset source
image_dir = '/content/data/data/train/images'
output_dir = '/content/banana_data'
os.makedirs(output_dir, exist_ok=True)

# Clean folder structure
for sub in ['Ripe_Banana', 'Raw_Banana']:
    for split in ['train', 'val', 'test']:
        os.makedirs(f"{output_dir}/{split}/{sub}", exist_ok=True)

# File separation
all_images = os.listdir(image_dir)
ripe_banana = [img for img in all_images if img.lower().startswith("ripe_banana")]
raw_banana = [img for img in all_images if img.lower().startswith("raw_banana")]

# Use equal number of images for balance
N = min(len(ripe_banana), len(raw_banana))
print(f"Using N = {N} samples per banana class")

# Split into train, val, test
def split_data(images, label_name):
    train, temp = train_test_split(images, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)
    for img in train:
        shutil.copy(os.path.join(image_dir, img), f"{output_dir}/train/{label_name}/{img}")
    for img in val:
        shutil.copy(os.path.join(image_dir, img), f"{output_dir}/val/{label_name}/{img}")
    for img in test:
        shutil.copy(os.path.join(image_dir, img), f"{output_dir}/test/{label_name}/{img}")

split_data(ripe_banana[:N], "Ripe_Banana")
split_data(raw_banana[:N], "Raw_Banana")

# Data generators
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(f"{output_dir}/train", target_size=(224, 224), batch_size=32, class_mode='categorical')
val_data = val_test_gen.flow_from_directory(f"{output_dir}/val", target_size=(224, 224), batch_size=32, class_mode='categorical')
test_data = val_test_gen.flow_from_directory(f"{output_dir}/test", target_size=(224, 224), batch_size=32, class_mode='categorical')

# Build model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_data, validation_data=val_data, epochs=10)

# Evaluate
loss, accuracy = model.evaluate(test_data)
print(f" Banana Classifier Test Accuracy: {accuracy * 100:.2f}%")

# Save model
model.save('banana_classifier_model.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('banana_classifier_model.tflite', 'wb') as f:
    f.write(tflite_model)

print(" Banana classifier training + test evaluation + TFLite conversion complete.")


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="banana_classifier_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get required input shape and dtype
input_shape = input_details[0]['shape']        # (1, 224, 224, 3)
input_dtype = input_details[0]['dtype']        # float32

# Class labels
class_labels = ["Raw_Banana", "Ripe_Banana"]

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224))
    img_normalized = img_resized.astype(np.float32) / 255.0
    input_tensor = np.expand_dims(img_normalized, axis=0).astype(input_dtype)
    return img_rgb, input_tensor

def classify_and_show(image_path):
    original_img, input_tensor = preprocess_image(image_path)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    # Get output tensor
    output = interpreter.get_tensor(output_details[0]['index'])[0]
    pred_idx = np.argmax(output)
    confidence = output[pred_idx]
    label = f"{class_labels[pred_idx]} ({confidence * 100:.2f}%)"

    # Draw label on image (red color)
    annotated_img = original_img.copy()
    cv2.putText(annotated_img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (255, 0, 0), 2, cv2.LINE_AA)  # Red = (255, 0, 0) in RGB

    # Show image in Colab
    plt.figure(figsize=(6,6))
    plt.imshow(annotated_img)
    plt.axis('off')
    plt.title("Prediction")
    plt.show()



 #RUN CLASSIFICATION
image_path = "/content/RIPE_BAN.jpg"  # Replace with your image
classify_and_show(image_path)




#  Example usage
image_paths = [
    "/content/RIPE_BAN.jpg",
    "/content/RW_BA3.jpg",
    "/content/RP_BA1.jpg",
    "/content/RP_BA2.jpg",
    "/content/RW_BA1.jpg"
]

for image_path in image_paths:
    classify_and_show(image_path)



In [ ]:
#Raspberry pi code
# Fruit Ripeness Classifier for Raspberry Pi using TFLite
# 1. Captures image using libcamera-still
# 2. Classifies with mango model
# 3. If Not_a_Mango or low confidence → uses banana model
# 4. Shows prediction on image + waits for manual close

import cv2
import numpy as np
import tflite_runtime.interpreter as tflite
import subprocess
import os

# --- TFLite model paths ---
MANGO_MODEL_PATH = "mango_classifier_model.tflite"
BANANA_MODEL_PATH = "banana_classifier_model.tflite"

# --- Class names ---
MANGO_CLASSES = ["Raw_Mango", "Ripe_Mango", "Not_a_Mango"]
BANANA_CLASSES = ["Raw_Banana", "Ripe_Banana"]

# --- Load models ---
def load_interpreter(model_path):
    interpreter = tflite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

mango_interpreter = load_interpreter(MANGO_MODEL_PATH)
banana_interpreter = load_interpreter(BANANA_MODEL_PATH)

# --- Preprocess image ---
def preprocess(frame):
    img = cv2.resize(frame, (224, 224))
    img = img.astype(np.float32) / 255.0
    return np.expand_dims(img, axis=0)

# --- Prediction function ---
def predict(interpreter, input_tensor, class_names):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]
    class_idx = np.argmax(output)
    confidence = float(output[class_idx])
    return class_names[class_idx], confidence

# --- Capture using libcamera ---
def capture_image():
    path = "/home/pi/captured.jpg"
    print(" Capturing image...")
    subprocess.run([
        "libcamera-still", "-o", path, "-t", "1000",
        "--width", "640", "--height", "480", "--nopreview"
    ], check=True)

    if not os.path.exists(path):
        raise RuntimeError("Image capture failed.")
    image = cv2.imread(path)
    if image is None:
        raise RuntimeError(" Could not read image.")
    return image

# --- Main classifier logic ---
def run_classifier():
    frame = capture_image()
    input_tensor = preprocess(frame)

    # Step 1: Mango model
    mango_label, mango_conf = predict(mango_interpreter, input_tensor, MANGO_CLASSES)
    print(f" Mango Model: {mango_label} ({mango_conf:.2f})")

    if mango_label == "Not_a_Mango" or mango_conf < 0.85:
        banana_label, banana_conf = predict(banana_interpreter, input_tensor, BANANA_CLASSES)
        label = banana_label
        confidence = banana_conf
        print(f"Banana Model: {label} ({confidence:.2f})")
    else:
        label = mango_label
        confidence = mango_conf

    #  Display result
    cv2.putText(frame, f"{label} ({confidence:.2f})", (10, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    print(f" Final Prediction: {label} ({confidence:.2f})")
    cv2.imshow("Fruit Classifier", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# --- Run program ---
if name == "main":
    run_classifier()